In [19]:
import torch
import cv2
import numpy as np
import pandas as pd
import math
from shapely.geometry import Point, Polygon  # 기하학적 물체 표현

road_x = np.array([[145, 720], [1250, 720], [1280, 605], [1280, 245], [1205, 245],
                   [1245, 320], [1170, 375], [920, 375], [675, 135], [615, 345],
                   [800, 430], [750, 500]])

ROI = Polygon(road_x)

road_x_ = np.array([[1280, 720], [1190, 365], [1025, 255], [185, 0], [115, 60],
                    [620, 185], [750, 255], [725, 330], [35, 530], [120, 635],
                    [730, 450], [970, 470]])

# 주정차 임의로 찍은거라 일단 위험지역은 다 주석처리 했음
# parking = [[305,850], [765,850], [765,1080], [305,1080]]
# center_parking = [(305+765)/2, (850+1080)/2]
# ROI_2 = Polygon([[12, 1132], [12, 2148], [3836, 2156], [3836, 888]])
# ROI_3 = Polygon([[12, 1132], [12, 2148], [3836, 2156], [3836, 888]])


# Calculate Homography
h_, status = cv2.findHomography(road_x, road_x_)
mppix = 4.2 / 54.08  # 1픽셀당 몇 미터인지 ==> 0.0352미터, 3/math.sqrt((1095-995)**2 + (480-545)**2)
# h_parking = [(h_[0][0] * center_parking[0] + h_[0][1] * center_parking[1] + h_[0][2]) / (h_[2][0] * center_parking[0] + h_[2][1] * center_parking[1] + 1),
#             (h_[1][0] * center_parking[0] + h_[1][1] * center_parking[1] + h_[1][2]) / (h_[2][0] * center_parking[0] + h_[2][1] * center_parking[1] + 1)]
h_parking = [745, 475]

# Model
# model = torch.hub.load('.', 'custom', path='./runs/train/exp11/weights/best.pt', source='local')
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

video_path = './school_main.mp4'

cap = cv2.VideoCapture(video_path)

if not cap.isOpened:
    print('--(!)Error opening video capture')
    sys.exit(1)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        break

    # Inference
    results = model(image)

    #     print(results.pandas().xyxy[0])
    #     for k in range(len(results.xyxy[0])):
    #         print(results.pandas().xyxy[0].name[k])
    #     break

    for k in range(len(results.xyxy[0])):
        conf = results.pandas().xyxy[0].confidence[0] * 100
        if conf >= 50:
            if results.pandas().xyxy[0].name[k] == 'person':
                label = "person: {:.2f}".format(conf)
                color_g = (0, 255, 0)
                cv2.rectangle(image, (int(results.pandas().xyxy[0].xmin[k]), int(results.pandas().xyxy[0].ymin[k])),
                              (int(results.pandas().xyxy[0].xmax[k]), int(results.pandas().xyxy[0].ymax[k])), color_g,
                              2)
                cv2.putText(image, label,
                            (int(results.pandas().xyxy[0].xmin[k]), int(results.pandas().xyxy[0].ymin[k] - 5)),
                            cv2.FONT_HERSHEY_DUPLEX, 1, color_g, 1)
            elif results.pandas().xyxy[0].name[k] == 'car':
                label = "car: {:.2f}".format(conf)
                color_y = (255, 255, 0)
                cv2.rectangle(image, (int(results.pandas().xyxy[0].xmin[k]), int(results.pandas().xyxy[0].ymin[k])),
                              (int(results.pandas().xyxy[0].xmax[k]), int(results.pandas().xyxy[0].ymax[k])), color_y,
                              2)
                cv2.putText(image, label,
                            (int(results.pandas().xyxy[0].xmin[k]), int(results.pandas().xyxy[0].ymin[k] - 5)),
                            cv2.FONT_HERSHEY_DUPLEX, 1, color_y, 1)
            elif results.pandas().xyxy[0].name[k] == 'bicycle':
                label = "bycycle: {:.2f}".format(conf)
                color_b = (0, 0, 255)
                cv2.rectangle(image, (int(results.pandas().xyxy[0].xmin[k]), int(results.pandas().xyxy[0].ymin[k])),
                              (int(results.pandas().xyxy[0].xmax[k]), int(results.pandas().xyxy[0].ymax[k])), color_b,
                              2)
                cv2.putText(image, label,
                            (int(results.pandas().xyxy[0].xmin[k]), int(results.pandas().xyxy[0].ymin[k] - 5)),
                            cv2.FONT_HERSHEY_DUPLEX, 1, color_b, 1)
            elif results.pandas().xyxy[0].name[k] == 'motorcycle':
                label = "motorcycle: {:.2f}".format(conf)
                color_p = (128, 0, 128)
                cv2.rectangle(image, (int(results.pandas().xyxy[0].xmin[k]), int(results.pandas().xyxy[0].ymin[k])),
                              (int(results.pandas().xyxy[0].xmax[k]), int(results.pandas().xyxy[0].ymax[k])), color_p,
                              2)
                cv2.putText(image, label,
                            (int(results.pandas().xyxy[0].xmin[k]), int(results.pandas().xyxy[0].ymin[k] - 5)),
                            cv2.FONT_HERSHEY_DUPLEX, 1, color_p, 1)
            else:
                continue

    # 기능 시작
    result = [0, 0, 0, 0, 0, 0, 0]  # 차도 사람, 자전거, 오토바이, 위험지역 사람, 자전거, 오토바이, 충돌위험
    in_driveway = []  # 차도 영역 내 객체 리스트, 차 <-> 오토바이,자전거랑 거리도 계산하게 했음
    car = []

    #    print(results.pandas().xyxy[0])

    for k in range(len(results.xyxy[0])):
        conf = results.pandas().xyxy[0].confidence[0] * 100
        if conf >= 50:
            # vehicle -> 박스 중점 / 나머지 -> 하단 중점
            if results.pandas().xyxy[0].name[k] == 'car':
                point = [(int(results.pandas().xyxy[0].xmin[k]) + int(results.pandas().xyxy[0].xmax[k])) // 2,
                         (int(results.pandas().xyxy[0].ymin[k]) + int(results.pandas().xyxy[0].ymax[k])) // 2]
                
                if point[0] > 640:
                    new_veh_x = (h_[0][0] * point[0] + h_[0][1] * point[1] + h_[0][2]) / (
                            h_[2][0] * point[0] + h_[2][1] * point[1] + 1)
                    new_veh_y = (h_[1][0] * point[0] + h_[1][1] * point[1] + h_[1][2]) / (
                            h_[2][0] * point[0] + h_[2][1] * point[1] + 1)

                    car.append([new_veh_x, new_veh_y])

            elif results.pandas().xyxy[0].name[k] == 'person':
                point = [(int(results.pandas().xyxy[0].xmin[k]) + int(results.pandas().xyxy[0].xmax[k])) // 2,
                          int(results.pandas().xyxy[0].ymax[k])]

                ### 차도 영역 내 보행자 판단
                ped = Point(point[0], point[1])
                new_ped_x = (h_[0][0] * point[0] + h_[0][1] * point[1] + h_[0][2]) / (
                        h_[2][0] * point[0] + h_[2][1] * point[1] + 1)
                new_ped_y = (h_[1][0] * point[0] + h_[1][1] * point[1] + h_[1][2]) / (
                        h_[2][0] * point[0] + h_[2][1] * point[1] + 1)

                if ped.within(ROI) == 1:
                    print("경고! 차도 내 보행자가 위치합니다!")
                    result[0] += 1
                    in_driveway.append([new_ped_x, new_ped_y])
#                 else:
#                     print("안전! 차도 내 보행자가 없습니다.")

                if math.sqrt(((new_ped_x - h_parking[0]) ** 2) + ((new_ped_y - h_parking[1]) ** 2)) * mppix <= 3:
                    print("경고! 위험구역 내 보행자가 위치합니다!")
                    result[3] += 1
#                 else:
#                     print("안전! 위험구역 내 보행자가 없습니다.")

            # 위험 지역 내 보행자 판단

            elif results.pandas().xyxy[0].name[k] == 'bicycle':
                point = [(int(results.pandas().xyxy[0].xmin[k]) + int(results.pandas().xyxy[0].xmax[k])) // 2,
                          int(results.pandas().xyxy[0].ymax[k])]

                ### 차도 영역 내 자전거 판단
                cyc = Point(point[0], point[1])
                new_cyc_x = (h_[0][0] * point[0] + h_[0][1] * point[1] + h_[0][2]) / (
                        h_[2][0] * point[0] + h_[2][1] * point[1] + 1)
                new_cyc_y = (h_[1][0] * point[0] + h_[1][1] * point[1] + h_[1][2]) / (
                        h_[2][0] * point[0] + h_[2][1] * point[1] + 1)
#                in_driveway.append([new_cyc_x, new_cyc_y])

                if cyc.within(ROI) == 1:
                    # print("경고! 차도 내 자전거가 위치합니다!")
                    result[1] += 1
#                 else:
#                     print("안전! 차도 내 자전거가 없습니다.")

                if math.sqrt(((new_cyc_x - h_parking[0]) ** 2) + ((new_cyc_y - h_parking[1]) ** 2)) * mppix <= 3:
                    print("경고! 위험구역 내 자전거가 위치합니다!")
                    result[4] += 1
#                 else:
#                     print("안전! 위험구역 내 자전거가 없습니다.")

                # 위험 지역 내 보행자 판단


            elif results.pandas().xyxy[0].name[k] == 'motorcycle':
                point = [(int(results.pandas().xyxy[0].xmin[k]) + int(results.pandas().xyxy[0].xmax[k])) // 2,
                          int(results.pandas().xyxy[0].ymax[k])]

                moto = Point(point[0], point[1])
                new_moto_x = (h_[0][0] * point[0] + h_[0][1] * point[1] + h_[0][2]) / (
                        h_[2][0] * point[0] + h_[2][1] * point[1] + 1)
                new_moto_y = (h_[1][0] * point[0] + h_[1][1] * point[1] + h_[1][2]) / (
                        h_[2][0] * point[0] + h_[2][1] * point[1] + 1)
#                 in_driveway.append([new_moto_x, new_moto_y])

                if moto.within(ROI) == 1:
                    print("경고! 차도 내 오토바이가 위치합니다!")
                    result[2] += 1
#                 else:
#                     print("안전! 차도 내 오토바이가 없습니다.")

                # 위험 지역 내 보행자 판단
                if math.sqrt(((new_moto_x - h_parking[0]) ** 2) + ((new_moto_y - h_parking[1]) ** 2)) * mppix <= 3:
                    print("경고! 위험구역 내 오토바이가 위치합니다!")
                    result[5] += 1
#                 else:
#                     print("안전! 위험구역 내 오토바이가 없습니다.")

    while car:
        ccar = car.pop()
        for i in range(len(in_driveway)):
            dis = math.sqrt(((in_driveway[i][0] - ccar[0]) ** 2) + ((in_driveway[i][1] - ccar[1]) ** 2)) * mppix
            if dis < 25:
                print("위험! 충돌 위험 객체가 있습니다! (위치 좌표 :", str(in_driveway[i][0]), ",", str(in_driveway[i][1]), ")")
                result[6] += 1
#                 else:
#                     print("안전! 충돌 위험 객체가 없습니다. (위치 좌표 :", str(ccar[0]), ",", str(ccar[1]), ")")

    print("@@@ result @@@", result)

    # Results
    cv2.imshow('Detection', image)

    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q') or key_pressed == 27:
        break

cap.release()
cv2.destroyAllWindows()

# 뒤에 서버 전송



Using cache found in C:\Users\SDH-LAB/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-7-22 Python-3.8.13 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1060 3GB, 3072MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 오토바이가 위치합니다!
경고! 위험구역 내 보행자가 위치합니다!
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 965.0886503562149 , 559.9815209094163 )
@@@ result @@@ [2, 0, 1, 1, 0, 0, 1]
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 오토바이가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 위험구역 내 보행자가 위치합니다!
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 973.6129427780759 , 562.503443206942 )
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 1246.4779041956858 , 371.85790836492856 )
@@@ result @@@ [3, 0, 1, 1, 0, 0, 2]
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 오토바이가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 985.6445511285762 , 563.713713925193 )
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 1246.5410106365193 , 371.37871192852845 )
@@@ result @@@ [3, 0, 1, 0, 0, 0, 2]
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 오토바이가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 976.5342688338486 , 568.6934499865208 )
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 1246.5410106365193 , 371.37871192852845 )
@@@ result @@@ [3, 0, 1, 0, 0, 0, 2]


경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
@@@ result @@@ [5, 1, 0, 0, 0, 0, 0]
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 위험구역 내 보행자가 위치합니다!
@@@ result @@@ [5, 1, 0, 1, 0, 0, 0]
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
@@@ result @@@ [4, 1, 0, 0, 0, 0, 0]
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
@@@ result @@@ [5, 0, 0, 0, 0, 0, 0]
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 405.40813785894284 , 132.56783767097602 )
@@@ result @@@ [4, 0, 0, 0, 0, 0, 1]
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
경고! 차도 내 보행자가 위치합니다!
위험! 충돌 위험 객체가 있습니다! (위치 좌표 : 411.04323656534075 , 130.83305455718408 )
@@@ result @@@ [5, 0, 0, 0, 0, 0, 1]
경고! 차도 내 보행자가 위치합니다!
경고! 